In [1]:
from sklearn.linear_model import LinearRegression
import pickle
import numpy as np
import os
from scipy.io import loadmat

curDir = '/home/yan/Data2/narvi_folder/RegressionFromMeasToPCAParams/NOMOfits/male/34D-Meas-20D-PCA/'
pkl_filename = curDir+'LinearRegression_model.pkl'

# load
with open(pkl_filename, 'rb') as file:
    regressor = pickle.load(file)
    
predDir = '/home/yan/Data2/narvi_folder/NOMO-Networks/HKS-Net/UF-US-2-TF/exp/exp_train_on_PCANOMOfits/male/PartMeasParams/clearBG/result/prediction/'

sampleList = os.listdir(predDir)
nSamples = len(sampleList)
print("totally %d samples"%nSamples)

totally 730 samples


In [2]:
pcaDir = '/home/yan/Data2/narvi_folder/Datasets/PCAModel-synth-on-NOMOfits/male/ori_pca_space/'

coeff = loadmat(pcaDir+'coeff.mat')
mu    = loadmat(pcaDir+'mu.mat')
face  = loadmat(pcaDir+'Face.mat')

coeff = coeff['coeff']
mu    = mu['mu']
face  = face['F']


coeff = np.asarray(coeff)
coeff = coeff[:, 0:20]
coeff = np.transpose(coeff)
print coeff.shape

mu = np.asarray(mu)
print mu.shape

face = np.asarray(face)
print face.shape

(20, 20670)
(1, 20670)
(13776, 3)


In [3]:
outDir = '/home/yan/Data2/narvi_folder/NOMO-Networks/HKS-Net/UF-US-2-TF/exp/exp_train_on_PCANOMOfits/male/PartMeasParams/clearBG/result/reconstruct_body_by_linearMapping/'

for idx in range(nSamples):
    sName = sampleList[idx][:-4]
    
    measPred = loadmat(predDir + sName + '.mat')
    measPred = measPred['pred'][0]

    pcaEst = regressor.predict(measPred.reshape(1, -1))
    pcaEst = pcaEst[0].reshape(1, -1)

    verts = np.matmul(pcaEst, coeff) + mu
    verts = verts.reshape(6890, 3)
    
    outMesh = outDir + sName + '.obj'
    with open(outMesh, 'w') as fp:
        for v in verts:
            fp.write('v %f %f %f\n'%(v[0], v[1], v[2]))
        for f in face:
            fp.write('f %d %d %d\n'%(f[0], f[1], f[2]))